In [ ]:
%matplotlib widget
import flammkuchen as fl
import lotr.plotting as pltltr
import numpy as np
import pandas as pd
from lotr import DATASET_LOCATION, LotrExperiment, dataset_folders
from lotr.data_preprocessing.dlc_tracking import export_dlc_behavior
from lotr.utils import convolve_with_tau, interpolate, nan_phase_jumps, zscore
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
from lotr import DATASET_LOCATION

fish_with_eyes = [f.parent for f in DATASET_LOCATION.glob("*/*/behavior_from_dlc.h5")]
fish_with_eyes

In [ ]:
def _process_reg(reg):
    reg = convolve_with_tau(reg, int(TAU_S * exp.fs))
    return zscore(reg)


def draw_train_test(n_pts, wnd):
    indexes = np.arange(n_pts - wnd, dtype=np.float)

    # Draw beginning of training window:
    train_i_start = np.random.randint(0, n_pts - wnd)
    train_i_end = train_i_start + wnd

    # Exclude training window from possible choiches:
    indexes[train_i_start:train_i_end] = np.nan
    # Exclude beginning and end if we are to close to limits:
    if train_i_start < wnd:
        indexes[:train_i_start] = np.nan
    if n_pts - train_i_end < wnd:
        indexes[train_i_end:] = np.nan

    indexes = indexes[~np.isnan(indexes)]  # .astype(np.int)
    test_i_start = int(np.random.choice(indexes))

    return slice(train_i_start, train_i_end), slice(test_i_start, test_i_start + wnd)

In [ ]:
TAU_S = 5

path = fish_with_eyes[0]
exp = LotrExperiment(path)
dlc_df = fl.load(path / "behavior_from_dlc.h5", "/data")

fictive_head = exp.fictive_heading
phase = exp.network_phase

eyes_arr = dlc_df["rt_eye_medfilt"] + dlc_df["lf_eye_medfilt"]
interp_eye = interpolate(dlc_df["t"], eyes_arr, exp.time_arr)

data_diff_df = pd.DataFrame(
    dict(
        phase=_process_reg(np.diff(np.unwrap(phase))),
        mov_regr=_process_reg(np.diff(fictive_head)),
        eye_pos_regr=_process_reg(interp_eye[1:]),
        eye_vel_regr=_process_reg(np.diff(interp_eye)),
    )
)
data_df = pd.DataFrame(
    dict(
        phase=np.unwrap(phase),
        mov_regr=_process_reg(fictive_head),
        eye_pos_regr=_process_reg(interp_eye),
    )
)

to_fit = data_diff_df

all_coefs = []
for _ in range(500):
    train, test = draw_train_test(len(phase), 300 * exp.fs)

    regr = LinearRegression()
    regr.fit(
        to_fit[["mov_regr", "eye_pos_regr"]].values[train, :],
        to_fit["phase"].values[train],
    )
    pred_fit = regr.predict(to_fit[["mov_regr", "eye_pos_regr"]].values[train, :])
    prediction = regr.predict(to_fit[["mov_regr", "eye_pos_regr"]].values[test, :])

    # pred_r2 = r2_score(prediction, to_fit["phase"][test])
    # train_r2 = r2_score(pred_fit, to_fit["phase"][train])

    train_corrs = np.stack(
        [
            to_fit["phase"][train],
            pred_fit,
            to_fit["mov_regr"][train] * regr.coef_[0],
            to_fit["eye_pos_regr"][train] * regr.coef_[1],
        ]
    )

    test_corrs = np.stack(
        [
            to_fit["phase"][test],
            prediction,
            to_fit["mov_regr"][test] * regr.coef_[0],
            to_fit["eye_pos_regr"][test] * regr.coef_[1],
        ]
    )

    all_coefs.append(
        np.stack([np.corrcoef(train_corrs)[0, 1:], np.corrcoef(test_corrs)[0, 1:]])
    )

all_coefs = np.array(all_coefs)

In [ ]:
b = np.arange(-1, 1, 0.05)
plt.figure()
for i in range(3):
    plt.hist(all_coefs[:, 0, i], b, alpha=0.3)

In [ ]:
f, axs = plt.subplots(2, 1, figsize=(6, 4))
axs[0].plot(
    to_fit[["mov_regr", "eye_pos_regr"]].values[train, :] * regr.coef_ + regr.intercept_
)
axs[0].plot(pred_fit)
axs[0].plot(to_fit["phase"][train].values)
axs[0].set_title(f"{train_r2:3.3f}, ({regr.coef_})")

axs[1].plot(
    to_fit[["mov_regr", "eye_pos_regr"]].values[test, :] * regr.coef_ + regr.intercept_
)
axs[1].plot(prediction)
axs[1].plot(to_fit["phase"][test].values)
axs[1].set_title(f"{pred_r2:3.3f}, ({regr.coef_})")

plt.tight_layout()

In [ ]:
dir(regr)

In [ ]:
plt.figure()
plt.plot(prediction)
plt.plot(data_df["phase"][train].values)
plt.plot(data_df.loc[:, ["mov_regr", "eye_pos_regr"]].values[train, :])

In [ ]:
f, axs = plt.subplots(figsize=(5, 5))
axs.scatter(
    data_df["eye_pos_regr"],
    data_df["mov_regr"],
    c=data_df["phase"],
    cmap="RdBu_r",
    s=5,
)

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(5, 5))
axs[0, 0].scatter(
    phase_diff, mov_regr, c=eye_pos_regr, vmin=-1.5, vmax=1.5, cmap="RdBu_r", s=5
)
axs[0, 1].scatter(
    phase_diff, mov_regr, c=eye_vel_regr, vmin=-1.5, vmax=1.5, cmap="RdBu_r", s=5
)
axs[1, 0].scatter(phase_diff, eye_pos_regr, c=mov_regr, cmap="RdBu_r", s=5)
axs[1, 1].scatter(phase_diff, eye_vel_regr, c=mov_regr, cmap="RdBu_r", s=5)

In [ ]:
plt.figure()
plt.plot(zscore(mov_regr))
plt.plot(zscore(eye_pos_regr))
plt.plot(zscore(eye_vel_regr))
# plt.plot(zscore(phase_diff))

In [ ]:
plt.figure()
plt.plot(exp.time_arr[1:], interp_eye)
plt.plot(dlc_df["t"], eyes_arr)